# ConvMixer Without PCA and PSA


## Mount Google Drive và chuyển đến thư mục làm việc

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/Fishclassify/'


Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1SbEBtvmNA96LIh5BdSkB7rPhTqtqlhP2/Fishclassify


In [2]:
import numpy as np
image_dir = '/content/drive/MyDrive/Fishclassify/data1/train'

In [3]:
import torchvision
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from sklearn import metrics


## Cài đặt và áp dụng các phép biến đổi cho dữ liệu hình ảnh

In [4]:
IMAGE_HEIGHT = 112
IMAGE_WIDTH = 112
train_transform = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.Rotate(limit=20, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.Normalize(mean=(0., 0., 0.), std=(1., 1., 1.), max_pixel_value=255.0),
    ToTensorV2(),
])


## Chia tập dữ liệu thành train và test

In [5]:
train_image_dir = 'data1/train'  # Đường dẫn đến thư mục chứa hình ảnh huấn luyện
test_image_dir = 'data1/test'    # Đường dẫn đến thư mục chứa hình ảnh kiểm tra

train_ds = torchvision.datasets.ImageFolder(train_image_dir, transform=None)
test_ds = torchvision.datasets.ImageFolder(test_image_dir, transform=None)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Định nghĩa class LoadData để tải dữ liệu

In [7]:
class LoadData(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __getitem__(self, idx):
        img, label = self.data[idx]
        img = np.asarray(img)
        if self.transform is not None:
            transformed = self.transform(image=img)
            img = transformed["image"]
        return img, label

    def __len__(self):
        return len(self.data)


## Tạo các DataLoader cho tập huấn luyện và tập kiểm tra

In [8]:
batch_size = 16

dataloaders = {
    'train': DataLoader(LoadData(train_ds, train_transform), batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0),
    'test': DataLoader(LoadData(test_ds, train_transform), batch_size=batch_size, shuffle=False, drop_last=True, num_workers=0),
}


## Định nghĩa mô hình ConvMixer



In [9]:
import torch.nn as nn

class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

def ConvMixer(dim, depth, kernel_size=9, patch_size=7, n_classes=12):
    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
            Residual(nn.Sequential(
                nn.Conv2d(dim, dim, kernel_size=kernel_size, groups=dim, padding="same"),
                nn.GELU(),
                nn.BatchNorm2d(dim)
            )),
            nn.Conv2d(dim, dim, kernel_size=1),
            nn.GELU(),
            nn.BatchNorm2d(dim)
        ) for i in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )


## Khởi tạo mô hình và định nghĩa hàm loss, optimizer

In [10]:
model = ConvMixer(dim=256, depth=8)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


## Huấn luyện mô hình

In [ ]:
num_epochs = 20
train_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloaders['train']:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_losses.append(running_loss / len(dataloaders['train']))
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloaders["train"])}')

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)

            correct += (predicted == labels).sum().item()

    test_accuracies.append(100 * correct / total)
    print(f'Accuracy: {100 * correct / total}%')

    # Lưu mô hình sau mỗi epoch
    torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')
torch.save(model.state_dict(), 'fish_classifier.pth')


Epoch 1/20, Loss: 1.9766306755494099
Accuracy: 25.76530612244898%
Epoch 2/20, Loss: 1.7425180795241375
Accuracy: 54.8469387755102%
Epoch 3/20, Loss: 1.584933030362032
Accuracy: 53.18877551020408%
Epoch 4/20, Loss: 1.2789278322336626
Accuracy: 43.23979591836735%
Epoch 5/20, Loss: 1.232765411844059
Accuracy: 66.70918367346938%
Epoch 6/20, Loss: 1.0184367140945123
Accuracy: 60.84183673469388%
Epoch 7/20, Loss: 0.9916225440648138
Accuracy: 69.7704081632653%
Epoch 8/20, Loss: 0.8825360232469986
Accuracy: 79.20918367346938%
Epoch 9/20, Loss: 0.7952143628986514
Accuracy: 77.93367346938776%
Epoch 10/20, Loss: 0.7623597270980174
Accuracy: 79.71938775510205%
Epoch 11/20, Loss: 0.6109067073890141
Accuracy: 77.16836734693878%
Epoch 12/20, Loss: 0.6647899461035826
Accuracy: 87.5%
Epoch 13/20, Loss: 0.5572972325037937
Accuracy: 84.6938775510204%


In [ ]:
import torch.nn as nn

class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

def ConvMixer(dim, depth, kernel_size=9, patch_size=7, n_classes=12):
    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
            Residual(nn.Sequential(
                nn.Conv2d(dim, dim, kernel_size=kernel_size, groups=dim, padding="same"),
                nn.GELU(),
                nn.BatchNorm2d(dim)
            )),
            nn.Conv2d(dim, dim, kernel_size=1),
            nn.GELU(),
            nn.BatchNorm2d(dim)
        ) for i in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

# Initialize the model here
model = ConvMixer(dim=256, depth=8)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# ... (rest of the training code) ...

## Vẽ đồ thị mất mát (loss) qua các epoch

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, marker='o', label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()


## Hiển thị độ chính xác (accuracy) qua các epoch

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), test_accuracies, marker='o', label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Test Accuracy Over Epochs')
plt.legend()
plt.show()


## Hiển thị một số kết quả dự đoán từ tập kiểm tra

In [ ]:
model.eval()
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(15, 6))
with torch.no_grad():
    for i in range(2):
        for j in range(5):
            idx = np.random.randint(len(test_ds))
            img, label = test_ds[idx]
            img_tensor = train_transform(image=np.asarray(img))['image'].unsqueeze(0)
            output = model(img_tensor)
            _, predicted = torch.max(output, 1)
            axes[i, j].imshow(img)
            axes[i, j].set_title(f'Pred: {predicted.item()}, True: {label}')
            axes[i, j].axis('off')
plt.show()


## Hiển thị ma trận nhầm lẫn (confusion matrix)

In [ ]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

# Tạo danh sách các nhãn thực tế và dự đoán
all_labels = []
all_predictions = []

model.eval()
with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Tạo ma trận nhầm lẫn
confusion_matrix = metrics.confusion_matrix(all_labels, all_predictions)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix=confusion_matrix)

# Hiển thị ma trận nhầm lẫn
plt.rcParams['figure.figsize'] = [12, 12]
cm_display.plot()
plt.xticks(rotation=60)
plt.yticks(rotation=0)
plt.show()


## Đây là ma trận nhầm lẫn dạng xịn hơn tí

In [ ]:
# !pip install seaborn
# import matplotlib.pyplot as plt
# import numpy as np
# import seaborn as sns
# from sklearn import metrics

# # Tạo danh sách các nhãn thực tế và dự đoán
# all_labels = []
# all_predictions = []

# model.eval()
# with torch.no_grad():
#     for inputs, labels in dataloaders['test']:
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs, 1)
#         all_labels.extend(labels.cpu().numpy())
#         all_predictions.extend(predicted.cpu().numpy())

# # Tạo ma trận nhầm lẫn
# confusion_matrix = metrics.confusion_matrix(all_labels, all_predictions)

# # Tạo biểu đồ nhiệt (heatmap) cho ma trận nhầm lẫn
# plt.figure(figsize=(12, 10))
# class_names = train_ds.classes
# sns.heatmap(confusion_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
# plt.xlabel('Predicted labels')
# plt.ylabel('True labels')
# plt.title('Confusion Matrix Heatmap')
# plt.xticks(rotation=45)
# plt.yticks(rotation=0)
# plt.show()


## Đoạn dưới đây lấy random cá ra nghịch

In [ ]:
import os
import random
from PIL import Image
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Đường dẫn tới thư mục chứa các tập tin trong Google Drive
# folder_path = '/content/drive/My Drive/Fishclassify/croatianFishDataset-final/Sarpa salpa'
folder_path = '/content/drive/MyDrive/Fishclassify/data1/test/Chromis chromis'

# Liệt kê tất cả các tập tin trong thư mục
files = os.listdir(folder_path)

# Chọn một tập tin ngẫu nhiên từ thư mục
selected_file = random.choice(files)
file_path = os.path.join(folder_path, selected_file)

# Đọc nội dung của tập tin vào biến X
image = Image.open(file_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Thay đổi kích thước hình ảnh nếu cần
    transforms.ToTensor(),  # Chuyển đổi hình ảnh thành tensor
])

X = transform(image)

# Thêm một chiều batch
X = X.unsqueeze(0)

# Nếu bạn muốn đưa tensor lên GPU (nếu có)
if torch.cuda.is_available():
    X = X.cuda()
    model = model.cuda()  # Đảm bảo mô hình cũng được đưa lên GPU nếu có

# Dự đoán: forward pass
model.eval()
with torch.no_grad():
    prob = model(X)

# Hàm hiển thị kết quả dự đoán
def show_prob(prob):
    prob = torch.nn.functional.softmax(prob, dim=1)
    prob = prob.cpu().numpy().flatten()
    class_indices = prob.argsort()[::-1]  # Sắp xếp theo xác suất giảm dần
    for i in range(len(class_indices)):
        print(f'Class {class_indices[i]}: {prob[class_indices[i]]:.4f}')
    return class_indices

# Hiển thị kết quả dự đoán
class_indices = show_prob(prob)

# Hiển thị hình ảnh đã dự đoán
plt.imshow(image)
plt.title(f'Predicted: Class {class_indices[0]}')
plt.axis('off')
plt.show()
